In [5]:
import numpy as np
import pandas as pd
import time
import concurrent.futures
import multiprocessing
from compare_packets import compare_packets_ratio, compare_packets_dist

pd.options.display.max_colwidth=200

In [ ]:
attack = False

if attack:
    #Fake attack
    df_attack = pd.read_csv("../checkpoints/URL_LSTM/11-29-2021/Attack/Spam_Samples_29-11-2021_11-38.csv", header=None, delimiter='\t')
    df_attack = df_attack.drop_duplicates(keep='first')
    fake_array=df_attack.to_numpy().squeeze()

    #real attack
    df1 = pd.read_csv("../Datasets/URL/FinalDataset/URL/spam_dataset.csv", header=None)
    df3 = pd.read_csv("../Datasets/URL/FinalDataset/URL/Malware_dataset.csv", header=None)
    df4 = pd.read_csv("../Datasets/URL/FinalDataset/URL/phishing_dataset.csv", header=None)
    df_attack_real = pd.concat([df1, df3, df4], ignore_index=True)

    real_samples = df_attack_real.to_numpy().squeeze()

else:
    #fake benign
    df_benign = pd.read_csv("../checkpoints/URL_LSTM/11-29-2021/Benign/Benign_Samples_29-11-2021_13-13.csv", header=None)
    df_benign = df_benign.drop_duplicates(keep='first')

    fake_array=df_benign.to_numpy().squeeze()

    #real benign
    df_benign_real = pd.read_csv("../Datasets/URL/FinalDataset/URL/Benign_list_big_final.csv")
    real_samples=df_benign_real.to_numpy().squeeze()
    print(real_samples[0])


print("Done Loading Data!")


In [7]:
print("CPU's available for multiprocessing:", multiprocessing.cpu_count())
cpu_cores = 18
fake_split = np.array_split(fake_array, cpu_cores)

#check lengths are correct
for lst in fake_split:
    print(len(lst))

def make_dist_lists(lists, ratio=True):
    lev_lists = []

    if ratio:
        val = 0
    else:
        val = np.inf

    for lst in lists:
        lst = np.array(lst)
        lev_dists = np.column_stack((lst, np.full_like(lst, fill_value=val), np.zeros_like(lst), np.zeros_like(lst)))
        lev_lists.append(lev_dists)

    return lev_lists

dist_lists = make_dist_lists(fake_split)

for lst in dist_lists:
    print(np.array(lst).shape)

real_lists = [real_samples for _ in range(cpu_cores)]

CPU's available for multiprocessing: 24
2717
2717
2716
2716
2716
2716
2716
2716
2716
2716
2716
2716
2716
2716
2716
2716
2716
2716
(2717, 4)
(2717, 4)
(2716, 4)
(2716, 4)
(2716, 4)
(2716, 4)
(2716, 4)
(2716, 4)
(2716, 4)
(2716, 4)
(2716, 4)
(2716, 4)
(2716, 4)
(2716, 4)
(2716, 4)
(2716, 4)
(2716, 4)
(2716, 4)


In [8]:
start = time.time()

with concurrent.futures.ProcessPoolExecutor() as executor:
    results = executor.map(compare_packets_ratio, fake_split, real_lists, dist_lists)

    res = []
    for lst in results:
        res.append(lst)
    np.save("lev_ratio_attack_URL", np.array(res))

end = time.time()
print("Elapsed = %s" % (end - start))


Elapsed = 1671.4427008628845


<ipython-input-8-2d98dc64c979>:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.save("lev_ratio_attack_URL", np.array(res))
